# Stock Screener Financial Metrics Test Notebook

This notebook demonstrates how to use the `data_fetcher` module to calculate key financial metrics for stock screening. Metrics include revenue growth, gross margin, P/E, ROE, ROAC, reinvestment rate, and payout ratio (both last year and historical average).

## 1. Import Required Libraries and data_fetcher

Import pandas, numpy, and the data_fetcher module. Set up logging if needed.

In [1]:
import pandas as pd
import numpy as np
import logging

# Import the data_fetcher module
import sys
sys.path.append(r"c:\Users\adamw\Desktop\stock_analysis\stock_screener")
from data_fetcher import get_historical_data

# Optional: Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("metrics_test")

## 2. Fetch Financial Data for a Stock

Use `get_historical_data` from data_fetcher to retrieve financial data and valuation info for a given ticker.

In [17]:
# Choose a ticker symbol (e.g., Apple)
ticker = "AAPL"

import yfinance as yf


ticker = yf.Ticker(ticker)



In [18]:
# Method 1: From .info (slower but more detailed)
pe_ratio = ticker.info.get("trailingPE")
print(f"Trailing P/E Ratio: {pe_ratio}")

# Method 2: From .fast_info (faster, but fewer fields)
fast_pe = ticker.fast_info.get("pe_ratio")
print(f"Fast P/E Ratio: {fast_pe}")

Trailing P/E Ratio: 33.314644
Fast P/E Ratio: None


## 3. Calculate Revenue Growth (Last Year & Historical Average)

Compute revenue growth for the last year and the historical average using the DataFrame returned by data_fetcher.

In [3]:
def revenue_growth(df):
    df_hist = df[df['ttm'] == 0].sort_index()
    rev = df_hist['Revenue'].dropna()
    # Last year growth
    if len(rev) >= 2:
        last_growth = (rev.iloc[-1] - rev.iloc[-2]) / rev.iloc[-2]
    else:
        last_growth = np.nan
    # Historical average CAGR
    if len(rev) >= 2:
        years = len(rev) - 1
        cagr = (rev.iloc[-1] / rev.iloc[0]) ** (1 / years) - 1
    else:
        cagr = np.nan
    return last_growth, cagr

last_rev_growth, avg_rev_growth = revenue_growth(df)
print(f"Last Year Revenue Growth: {last_rev_growth:.2%}")
print(f"Historical Average Revenue Growth (CAGR): {avg_rev_growth:.2%}")

Last Year Revenue Growth: 2.02%
Historical Average Revenue Growth (CAGR): 2.25%


## 4. Calculate Gross Margin (Last Year & Historical Average)

Calculate gross margin for the last year and historical average using revenue and gross profit (if available).

In [4]:
def gross_margin(df):
    df_hist = df[df['ttm'] == 0].sort_index()
    # If Gross Profit is available
    if 'Gross Profit' in df_hist.columns:
        gross = df_hist['Gross Profit'].dropna()
        rev = df_hist['Revenue'].dropna()
        margin = gross / rev
    else:
        margin = pd.Series([np.nan] * len(df_hist), index=df_hist.index)
    # Last year and average
    last_margin = margin.iloc[-1] if len(margin) > 0 else np.nan
    avg_margin = margin.mean() if len(margin) > 0 else np.nan
    return last_margin, avg_margin

last_gross_margin, avg_gross_margin = gross_margin(df)
print(f"Last Year Gross Margin: {last_gross_margin:.2%}")
print(f"Historical Average Gross Margin: {avg_gross_margin:.2%}")

Last Year Gross Margin: nan%
Historical Average Gross Margin: nan%


## 5. Calculate Price/Earnings Ratio (P/E) (Last Year & Historical Average)

Calculate P/E ratio for the last year and historical average using share price and net income.

In [ ]:
def pe_ratio(df, valuation_info):
    df_hist = df[df['ttm'] == 0].sort_index()
    net_income = df_hist['Net Income'].dropna()
    shares = valuation_info.get("number_shares", np.nan)
    price = valuation_info.get("share_price", np.nan)
    # EPS for each year
    eps = net_income / shares if pd.notna(shares) and shares != 0 else pd.Series([np.nan]*len(net_income), index=net_income.index)
    pe = price / eps if pd.notna(price) else pd.Series([np.nan]*len(eps), index=eps.index)
    last_pe = pe.iloc[-1] if len(pe) > 0 else np.nan
    avg_pe = pe.mean() if len(pe) > 0 else np.nan
    return last_pe, avg_pe

last_pe, avg_pe = pe_ratio(df, valuation_info)
print(f"Last Year P/E Ratio: {last_pe:.2f}")
print(f"Historical Average P/E Ratio: {avg_pe:.2f}")

## 6. Calculate Return on Equity (ROE) (Last Year & Historical Average)

Calculate ROE for the last year and historical average using net income and equity.

In [ ]:
def roe(df):
    df_hist = df[df['ttm'] == 0].sort_index()
    net_income = df_hist['Net Income'].dropna()
    # Equity = Total Assets - Total Debt
    if 'Total Assets' in df_hist.columns and 'Total Debt' in df_hist.columns:
        equity = df_hist['Total Assets'] - df_hist['Total Debt']
        roe_series = net_income / equity
    else:
        roe_series = pd.Series([np.nan]*len(net_income), index=net_income.index)
    last_roe = roe_series.iloc[-1] if len(roe_series) > 0 else np.nan
    avg_roe = roe_series.mean() if len(roe_series) > 0 else np.nan
    return last_roe, avg_roe

last_roe, avg_roe = roe(df)
print(f"Last Year ROE: {last_roe:.2%}")
print(f"Historical Average ROE: {avg_roe:.2%}")

## 7. Calculate Return on Assets/Capital (ROAC) (Last Year & Historical Average)

Calculate ROAC for the last year and historical average using EBIT and capital invested.

In [ ]:
def roac(df):
    df_hist = df[df['ttm'] == 0].sort_index()
    ebit = df_hist['EBIT'].dropna()
    capital_invested = df_hist['capital_invested'].dropna() if 'capital_invested' in df_hist.columns else pd.Series([np.nan]*len(ebit), index=ebit.index)
    roac_series = ebit / capital_invested
    last_roac = roac_series.iloc[-1] if len(roac_series) > 0 else np.nan
    avg_roac = roac_series.mean() if len(roac_series) > 0 else np.nan
    return last_roac, avg_roac

last_roac, avg_roac = roac(df)
print(f"Last Year ROAC: {last_roac:.2%}")
print(f"Historical Average ROAC: {avg_roac:.2%}")

## 8. Calculate Reinvestment Rate (Last Year & Historical Average)

Calculate reinvestment rate for the last year and historical average using CapEx, depreciation, delta WC, and NOPAT.

In [ ]:
def reinvestment_rate(df):
    df_hist = df[df['ttm'] == 0].sort_index()
    if 'ReinvestmentRate' in df_hist.columns:
        reinvest = df_hist['ReinvestmentRate'].dropna()
        last_reinvest = reinvest.iloc[-1] if len(reinvest) > 0 else np.nan
        avg_reinvest = reinvest.mean() if len(reinvest) > 0 else np.nan
    else:
        last_reinvest = np.nan
        avg_reinvest = np.nan
    return last_reinvest, avg_reinvest

last_reinvest, avg_reinvest = reinvestment_rate(df)
print(f"Last Year Reinvestment Rate: {last_reinvest:.2%}")
print(f"Historical Average Reinvestment Rate: {avg_reinvest:.2%}")

## 9. Calculate Payout Ratio (Last Year & Historical Average)

Calculate payout ratio for the last year and historical average using dividends and net income.

In [ ]:
def payout_ratio(df):
    df_hist = df[df['ttm'] == 0].sort_index()
    # If Dividends Paid is available
    if 'Dividends Paid' in df_hist.columns:
        dividends = df_hist['Dividends Paid'].dropna()
        net_income = df_hist['Net Income'].dropna()
        payout = dividends / net_income
    else:
        payout = pd.Series([np.nan]*len(df_hist), index=df_hist.index)
    last_payout = payout.iloc[-1] if len(payout) > 0 else np.nan
    avg_payout = payout.mean() if len(payout) > 0 else np.nan
    return last_payout, avg_payout

last_payout, avg_payout = payout_ratio(df)
print(f"Last Year Payout Ratio: {last_payout:.2%}")
print(f"Historical Average Payout Ratio: {avg_payout:.2%}")